In [1]:
import sys
import pandas as pd
import xlwings as xw
import openpyxl
import pyxlsb

print(f"Python: {sys.version.split()[0]}")
print(f"xlwings: {xw.__version__}")
print(f"OpenPyXL: {openpyxl.__version__}")
print(f"pyxlsb: {pyxlsb.__version__}")
print(f"pandas: {pd.__version__}")

Python: 3.10.6
xlwings: 0.28.3
OpenPyXL: 3.0.10
pyxlsb: 1.0.10
pandas: 1.5.1


## xlwings: Interactive mode (default)

In [2]:
book = xw.Book("AAPL.xlsx")
sheet = book.sheets[0]
df1 = sheet["A1:G10"].options("df").value
df1

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1980-12-12,0.128348,0.128906,0.128348,0.128348,0.100039,469033600.0
1980-12-15,0.122210,0.122210,0.121652,0.121652,0.094820,175884800.0
1980-12-16,0.113281,0.113281,0.112723,0.112723,0.087861,105728000.0
1980-12-17,0.115513,0.116071,0.115513,0.115513,0.090035,86441600.0
1980-12-18,0.118862,0.119420,0.118862,0.118862,0.092646,73449600.0
1980-12-19,0.126116,0.126674,0.126116,0.126116,0.098300,48630400.0
1980-12-22,0.132254,0.132813,0.132254,0.132254,0.103084,37363200.0
1980-12-23,0.137835,0.138393,0.137835,0.137835,0.107434,46950400.0
1980-12-24,0.145089,0.145647,0.145089,0.145089,0.113088,48003200.0


In [3]:
book.app.quit()

## xlwings: Read mode

In [4]:
with xw.Book("AAPL.xlsx", mode="r") as book:
    sheet = book.sheets[0]
    df2 = sheet["A1:G10"].options("df").value
df2

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1980-12-12,0.128348,0.128906,0.128348,0.128348,0.100039,469033600.0
1980-12-15,0.122210,0.122210,0.121652,0.121652,0.094820,175884800.0
1980-12-16,0.113281,0.113281,0.112723,0.112723,0.087861,105728000.0
1980-12-17,0.115513,0.116071,0.115513,0.115513,0.090035,86441600.0
1980-12-18,0.118862,0.119420,0.118862,0.118862,0.092646,73449600.0
1980-12-19,0.126116,0.126674,0.126116,0.126116,0.098300,48630400.0
1980-12-22,0.132254,0.132813,0.132254,0.132254,0.103084,37363200.0
1980-12-23,0.137835,0.138393,0.137835,0.137835,0.107434,46950400.0
1980-12-24,0.145089,0.145647,0.145089,0.145089,0.113088,48003200.0


In [5]:
df1.equals(df2)

True

# Speed

### pandas (via OpenPyXL)

In [6]:
%%time
df_xlsx = pd.read_excel("AAPL.xlsx", sheet_name=0, index_col="Date")

CPU times: user 801 ms, sys: 16.1 ms, total: 818 ms
Wall time: 855 ms


### xlwings

In [7]:
%%time
with xw.Book("AAPL.xlsx", mode="r") as book:
    sheet = book.sheets[0]
    df_xw = sheet.cells.options("df").value

CPU times: user 79.6 ms, sys: 5.57 ms, total: 85.2 ms
Wall time: 101 ms


### Check if all DataFrames are equal

In [8]:
df_xlsx["Volume"] = df_xlsx["Volume"].astype(float)
df_xlsx.equals(df_xw)

True

# Even faster with xlsb!

### pandas (via pyxlsb)

In [9]:
%%time
df_xlsb = pd.read_excel("AAPL.xlsb", sheet_name=0, index_col="Date")

CPU times: user 576 ms, sys: 8.16 ms, total: 584 ms
Wall time: 591 ms


### xlwings

In [10]:
%%time
with xw.Book("AAPL.xlsb", mode="r") as book:
    df_xlsb_xw = book.sheets[0].cells.options("df").value

CPU times: user 23.7 ms, sys: 4.83 ms, total: 28.5 ms
Wall time: 38.8 ms


### Check if all DataFrames are equal

In [11]:
df_xlsb.index = df_xlsb.index.astype(float)
df_xlsb["Volume"] = df_xlsb["Volume"].astype(float)
df_xlsb_xw.equals(df_xlsb)

True

# Syntax

### pandas

In [12]:
df = pd.read_excel(
    "AAPL2.xlsx",
    sheet_name=0,
    skiprows=10,
    usecols="B:G",
    nrows=2,
    index_col="Date"
)
df

,Open,High,Low,Close,Adj Close
Date,,,,,
1980-12-12,0.513393,0.515625,0.513393,0.513393,0.405683
1980-12-15,0.488839,0.488839,0.486607,0.486607,0.384517


### OpenPyXL

In [13]:
book = openpyxl.load_workbook(
    "AAPL2.xlsx", read_only=True, data_only=True
)
sheet = book.worksheets[0]
data = [
    row
    for row in sheet.iter_rows(
        min_row=11, max_row=13,
        min_col=2, max_col=7,
        values_only=True
    )
]
pd.DataFrame(data=data[1:], columns=data[0]).set_index("Date")

,Open,High,Low,Close,Adj Close
Date,,,,,
1980-12-12,0.513393,0.515625,0.513393,0.513393,0.405683
1980-12-15,0.488839,0.488839,0.486607,0.486607,0.384517


### xlwings (A1 notation)

In [14]:
with xw.Book("AAPL2.xlsx", mode="r") as book:
    df = book.sheets[0]["B11:G13"].options("df").value
df

,Open,High,Low,Close,Adj Close
Date,,,,,
1980-12-12,0.513393,0.515625,0.513393,0.513393,0.405683
1980-12-15,0.488839,0.488839,0.486607,0.486607,0.384517


### xlwings (slice notation)

In [15]:
with xw.Book("AAPL2.xlsx", mode="r") as book:
    df = book.sheets[0][10:13, 1:7].options("df").value
df

,Open,High,Low,Close,Adj Close
Date,,,,,
1980-12-12,0.513393,0.515625,0.513393,0.513393,0.405683
1980-12-15,0.488839,0.488839,0.486607,0.486607,0.384517


# Named Ranges

In [16]:
# Direct access
with xw.Book("AAPL2.xlsx", mode="r") as book:
    sheet = book.sheets[0]
    df = sheet["statistics"].options("df").value
df

,Value
Statistic,
Enterprise Value,2.42T
Trailing P/E,2.49T
Forward P/E,24.86


In [17]:
# Names collection
with xw.Book("AAPL2.xlsx", mode="r") as book:
    for name in book.names:
        print(name.name)
        print(name.refers_to_range)
        print(name.refers_to_range.sheet.name)
        print(name.refers_to_range.options("df").value)

statistics
<Range [AAPL2.xlsx]AAPL!$B$4:$C$7>
AAPL
                  Value
Statistic              
Enterprise Value  2.42T
Trailing P/E      2.49T
Forward P/E       24.86


# Not everything is a DataFrame

In [18]:
with xw.Book("AAPL2.xlsx", mode="r") as book:
    sheet = book.sheets[0]
    myvalue = sheet["B2"].value
    mylist = sheet["B4:C4"].value
    # Dynamic range expansion!
    mydict = sheet["B5"].expand().options(dict).value

In [19]:
myvalue

'Apple Inc. (AAPL)'

In [20]:
mylist

['Statistic', 'Value']

In [21]:
mydict

{'Enterprise Value': '2.42T', 'Trailing P/E': '2.49T', 'Forward P/E': 24.86}